In [ ]:
import xarray as xr
import cfgrib
%pylab inline
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import warnings
warnings.filterwarnings('ignore')
from wrf import smooth2d
import matplotlib.transforms as mtransforms

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.
G = 9.81

def get_t(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'r'}})
    var = file.get('r')[5].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var


In [ ]:
min_lat1 = 20
max_lat1 = 50
min_lon1 = -100
max_lon1 = -60

fcs = get_t('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f060.grib2',
                   min_lat1, max_lat1, min_lon1, max_lon1)

anl = get_t('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111600.f000.grib2',
                   min_lat1, max_lat1, min_lon1, max_lon1)


In [ ]:
fig, axd = plt.subplot_mosaic([['left'],['cbar1']],
                              constrained_layout=True, figsize=(5,4), dpi=1000, 
                              gridspec_kw={'width_ratios':[1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':crs.PlateCarree()}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

axd['left'].set_extent((min_lon1,max_lon1,min_lat1,max_lat1-2))


im1 = axd['left'].contourf(anl['longitude'], anl['latitude'], fcs['r'] - anl['r'],
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*6,extend='both',
                  cmap=get_cmap('BrBG'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-95, -85, -75, -65])
gls.ylocator = mticker.FixedLocator([25, 35, 45])

axd['left'].set_title('0000 UTC 16 Nov 2018 ($t$ = 60 h)')

### SET THE COLORBAR AND SHOW
cbar1 = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar1'])
plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/drh850_gfs.png',bbox_inches='tight')

plt.show()